In [11]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [12]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [60]:
def read_file_in_chunks(filename, chunk_size=2000):
    with open(filename, 'r', encoding='utf-8') as f:
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            yield chunk

def process_with_chatgpt_api(chunk, chatgpt_api_func):
    # Here you would call your chatgpt api function with the chunk as input.
    response = chatgpt_api_func(chunk)
    return response

In [63]:
filename = './article.txt'
chunks = list(read_file_in_chunks(filename))
length = len(chunks)

messages =  [  
{'role':'system', 'content':'You are an technology article professional translater at translating article from English to zh-hant-tw.'},
{'role':'assistant', 'content':'Ok, I am a professional translator from English to zh-hant-tw.'},
{'role':'user', 'content':f'I will provide you {length} passages, which is from a article as follows.'},
{'role':'assistant', 'content':f'Sure, I will read through {length} passages and recognize it as one article.'}, 
]

for i, chunk in enumerate(chunks):
    messages.append({'role':'user', 'content':f'Number of passage: {i+1}/{length} \n\n Content: \n {chunk}'})

messages.append({'role':'user', 'content':f"""
You have already being provided a article separated in {length} passages, it is most likely a technology article, \
but some times it will be something other than that.

You have to
1. Translate the article into ZH-HANT-TW
2. Reorganize the zh-hant-tw article into markdown raw format with proper highlighting.
3. Output the trnaslated zh-hant-tw article in markdown format.
"""})

In [64]:
from datetime import datetime

def write_to_markdown(text):
    # Get the current date
    current_date = datetime.now()

    # Format the date as a string in the format "yymmdd"
    date_string = current_date.strftime('%y%m%d')

    # Create the filename
    filename = f"{date_string}-Stratechery.md"

    # Write the text to the file
    with open(filename, 'w', encoding='utf-8') as f:
        f.write(text)      

responses = []
response = get_completion_from_messages(messages, temperature=0)
responses.append(response)
messages.append({'role':'assistant', 'content':f'{response}'})
messages.append({'role':'user', 'content':'continue the message'})
response = get_completion_from_messages(messages, temperature=0)
responses.append(response)
response = ''.join(responses)
write_to_markdown(response)